# Czyszczenie danych i przygotowanie zbioru do analizy

Pracę można wykonać w R lub Python (rekomendowany Python+pandas/numpy)

Pobierz zbiór danych TitanicMess.tsv. Jest to plik którego separatorem jest tabulacja
Przygotuj skoroszyt Jupyter - w którym będziesz dokumentować poszczególne kroki analizy a następnie czyszczenia zbioru
Zapoznaj się ze zbiorem - przyglądnij mu się. Zidentyfikuj i opisz problemy z jakością danych które znalazłeś (aś) w poszczególnych kolumnach, w całym zbiorze. Opisz jakie problemy zidentyfikowałas(eś) i pokaz gdzie je widać albo na podstawie czego o nich wnioskujesz.
Wyczyść zbiór, tzn usuń problemy które znalazłeś/aś. Opisz krótko co i dlaczego robisz - i jaki efekt masz nadzieję osiągnąć. No i .. pokaż że się udało, bo problemu już nie ma.
Zapisz oczyszczony zbiór pod nazwą TitanicCleaned.tsv

*Do rozpoczęcia pracy nad zbiorem wymagane jest zaimportowanie bibliotek podstawowych do tworzenia funkcjnalności.<br>
W tym sprawozdaniu wykorzystano biblioteki numpy, pandas i math.*

In [92]:
import numpy as np
import pandas as pd
import math
import locale
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

'en_US.UTF-8'

Kolejny etap to wczytanie zbioru i sprawdzenie poprawności importu.<br>
Ustawiono **znak tabulacji** jako separator.<br>
Ustawiono **PassengerID** jako kolumnę indeksującą.

In [93]:
titanic_df = pd.read_csv('TitanicMess.tsv', sep='\t', index_col='PassengerId', decimal=',')
titanic_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic


Kolejny etap to zidentyfikowanie kolumn z błędnymi lub pustymi wartościami.

In [94]:
titanic_df.dtypes

Survived     int64
Pclass       int64
Name        object
Sex         object
Age         object
SibSp        int64
Parch        int64
Ticket      object
Fare        object
Cabin       object
Embarked    object
ship        object
dtype: object

Wynik powyższego bloku kodu pozwala na wywnioskowanie, że prawdopodobnie kolumny Age i Fare zawierają błędne dane, gdyż nazwa sugeruje wartości liczbowe, a typ jest oznaczony jako object.<br>
Teraz sprawdzona zostanie ilość wartości pustych.

In [95]:
titanic_df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         173
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64

Kolumny Age, Cabin oraz Embarked zawierają puste wartości.

Przeglądając zbiór danych można zauważyć, że wszystkie rekordy w kolumnie Ship zawierają wartość *Titanic*. <br>
Pozwala to na usunięcie kolumny w celu optymalizacji, gdyż cały zbiór dotyczy tylko statku Titanic, więc jest ona zwyczajnie zbędna.

In [96]:
titanic_clear_columns = titanic_df.drop(columns='ship')
titanic_clear_columns.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S


Głębsza analiza zbioru danych pozwala na wysunięcie kolejnych wniosków:<br>
- **Survived** - cecha wyjściowa, gdzie 1 oznacza, że pasażer przeżył katastrofę, a 0 - nie
- **Pclass** - klasa pasażera (1,2,3)
- **Age** - wiek pasażera

Wywnioskowano z wcześniejszej analizy, że kolumna **Age** zawiera puste wartości.<br>
Postanowiono wypełnić je wartościami średnimi z całej kolumny, co umożliwi biblioteka PANDAS.

In [97]:
titanic_clear_columns['Age'].mean()

TypeError: can only concatenate str (not "int") to str

Wywołanie metody wyciągającej średnią (mean()) powoduje błąd. Spowodowane jest to faktem, że kolumna zawiera wartości innego typu niż numeryczne.<br>
Należy znaleźć błędne rekordy poprzez blok try-catch.

In [98]:
wrongAges=0 ## number of wrong fields
for index, row in titanic_clear_columns.iterrows():
    try:
        float(row['Age'])  ## parsing to float
    except ValueError:
        wrongAges = wrongAges + 1
        print("Wrong age value for passegnerId " + str(index))
    
print("Number of not numerical age values " + str(wrongAges)) 

Wrong age value for passegnerId 58
Wrong age value for passegnerId 79
Wrong age value for passegnerId 112
Wrong age value for passegnerId 117
Wrong age value for passegnerId 123
Wrong age value for passegnerId 124
Wrong age value for passegnerId 149
Wrong age value for passegnerId 153
Wrong age value for passegnerId 154
Wrong age value for passegnerId 204
Wrong age value for passegnerId 228
Wrong age value for passegnerId 297
Wrong age value for passegnerId 306
Wrong age value for passegnerId 332
Wrong age value for passegnerId 470
Wrong age value for passegnerId 526
Wrong age value for passegnerId 645
Wrong age value for passegnerId 677
Wrong age value for passegnerId 736
Wrong age value for passegnerId 756
Wrong age value for passegnerId 768
Wrong age value for passegnerId 804
Wrong age value for passegnerId 815
Wrong age value for passegnerId 832
Wrong age value for passegnerId 844
Number of not numerical age values 25


Należy przekonwertować powyższe wartości do typu float lub jeśli niemożliwe, to ustawienie wartości NaN.<br>
Zakres wieku ustawiono na 0-123 (123 to wiek najdłużej żyjącego człowieka w historii).<br>
Po wykonaniu powyższych instrukcji nastąpiło sprawdzenie typu kolumn,  w celu potwierdzenia poprawności operacji.

In [99]:
for i, row in titanic_clear_columns.iterrows():
    try:
        if float(row["Age"]) < 0 or float(row["Age"]) > 123:
            titanic_clear_columns.at[i, "Age"] = np.NaN
        else:
            titanic_clear_columns.at[i, "Age"] = float(row["Age"])
        
    except ValueError:
        titanic_clear_columns.at[i, "Age"] = np.NaN
for index, row in titanic_clear_columns.iterrows():
    print(row["Age"])

titanic_clear_columns[["Age"]] = titanic_clear_columns[["Age"]].astype(float).round(1) ## parsing to float
titanic_clear_columns.dtypes ## validate data


22.0
38.0
26.0
35.0
35.0
nan
54.0
2.0
27.0
14.0
4.0
58.0
20.0
4.0
14.0
55.0
2.0
nan
31.0
nan
35.0
34.0
15.0
4.0
8.0
38.0
nan
19.0
nan
nan
40.0
0.9
nan
66.0
28.0
42.0
nan
21.0
18.0
14.0
40.0
27.0
nan
3.0
19.0
nan
nan
nan
nan
18.0
7.0
21.0
49.0
29.0
65.0
nan
21.0
nan
5.0
11.0
22.0
38.0
45.0
4.0
nan
nan
29.0
19.0
17.0
26.0
32.0
16.0
21.0
26.0
32.0
25.0
nan
nan
nan
30.0
22.0
29.0
nan
28.0
17.0
33.0
16.0
nan
23.0
24.0
29.0
20.0
46.0
26.0
59.0
nan
71.0
23.0
34.0
34.0
28.0
nan
21.0
33.0
37.0
28.0
21.0
nan
38.0
nan
47.0
nan
22.0
20.0
17.0
21.0
nan
29.0
24.0
2.0
21.0
nan
nan
nan
54.0
12.0
nan
24.0
nan
45.0
33.0
20.0
47.0
29.0
25.0
23.0
19.0
37.0
16.0
24.0
nan
22.0
24.0
19.0
18.0
19.0
27.0
9.0
nan
42.0
51.0
22.0
nan
nan
nan
51.0
16.0
30.0
nan
nan
44.0
40.0
26.0
17.0
1.0
9.0
nan
45.0
nan
28.0
61.0
4.0
1.0
21.0
56.0
18.0
nan
50.0
30.0
36.0
nan
nan
9.0
1.0
4.0
nan
nan
45.0
40.0
36.0
32.0
19.0
19.0
3.0
44.0
58.0
nan
42.0
nan
24.0
28.0
nan
34.0
nan
18.0
2.0
32.0
26.0
16.0
40.0
24.0
35.0
22.0
30.0
nan

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare         object
Cabin        object
Embarked     object
dtype: object

Powyższy wynik przedstawia, że operacja zakończyła się sukcesem, co pozwala na wyciągnięcie i zaokrąglenie średniego wieku pasażera.

In [100]:
round(titanic_clear_columns['Age'].mean())

30

Obliczoną wartość wstawiono do pól pustych i błędnych.

In [101]:
titanic_clear_columns['Age'] = titanic_clear_columns['Age'].fillna(round(titanic_clear_columns['Age'].mean()))
for index, row in titanic_clear_columns.iterrows():
    print(row["Age"])

22.0
38.0
26.0
35.0
35.0
30.0
54.0
2.0
27.0
14.0
4.0
58.0
20.0
4.0
14.0
55.0
2.0
30.0
31.0
30.0
35.0
34.0
15.0
4.0
8.0
38.0
30.0
19.0
30.0
30.0
40.0
0.9
30.0
66.0
28.0
42.0
30.0
21.0
18.0
14.0
40.0
27.0
30.0
3.0
19.0
30.0
30.0
30.0
30.0
18.0
7.0
21.0
49.0
29.0
65.0
30.0
21.0
30.0
5.0
11.0
22.0
38.0
45.0
4.0
30.0
30.0
29.0
19.0
17.0
26.0
32.0
16.0
21.0
26.0
32.0
25.0
30.0
30.0
30.0
30.0
22.0
29.0
30.0
28.0
17.0
33.0
16.0
30.0
23.0
24.0
29.0
20.0
46.0
26.0
59.0
30.0
71.0
23.0
34.0
34.0
28.0
30.0
21.0
33.0
37.0
28.0
21.0
30.0
38.0
30.0
47.0
30.0
22.0
20.0
17.0
21.0
30.0
29.0
24.0
2.0
21.0
30.0
30.0
30.0
54.0
12.0
30.0
24.0
30.0
45.0
33.0
20.0
47.0
29.0
25.0
23.0
19.0
37.0
16.0
24.0
30.0
22.0
24.0
19.0
18.0
19.0
27.0
9.0
30.0
42.0
51.0
22.0
30.0
30.0
30.0
51.0
16.0
30.0
30.0
30.0
44.0
40.0
26.0
17.0
1.0
9.0
30.0
45.0
30.0
28.0
61.0
4.0
1.0
21.0
56.0
18.0
30.0
50.0
30.0
36.0
30.0
30.0
9.0
1.0
4.0
30.0
30.0
45.0
40.0
36.0
32.0
19.0
19.0
3.0
44.0
58.0
30.0
42.0
30.0
24.0
28.0
30.0
34.0
30.0
1

Kolejnym etapem będzie usunięcie kolumny **Cabin**.<br>
W wyniku analizy ustalono, że wartości kolumny to w większości wartości zerowe, poza tym nie przynoszą one żadnej pożytecznej wiedzy, dodatkowo będą negatywnie wpływały na optymalizację uczenia maszynowego.

In [102]:
titanic_clear_columns.drop('Cabin',axis=1, inplace=True)
titanic_clear_columns.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,"7,25",S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,"71,2833",C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,"7,925",S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,"53,1",S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,"8,05",S


Kolejnym etapem będzie uzupełnienie brakujących wartości kolumny **Embarked**.<br>
Jako że brakujących wartości jest tylko 2, wybrano losowo wartość 'S' i wpisano do brakujących pól.

In [103]:
titanic_clear_columns['Embarked'] = titanic_clear_columns['Embarked'].fillna('S')
titanic_clear_columns.isnull().values.any()

False

Wynik 'False" ostatniego bloku oznacza, że kolumna 'Embarked' nie zawiera już pustych wartości.

Kolejnym etapem będzie sprawdzenie unikalnych wartości kolumny **Sex**.<br>

In [104]:
unkalna_plec = titanic_clear_columns['Sex'].unique()
print(unkalna_plec)

['male' 'female' 'malef' 'mal' 'fem' 'femmale']


Powyższy wynik podsuwa wniosek, że w danych wystąpiły literówki. Zamieniono więc błędne wartości na poprawne, najbliżej brzmiące.

In [105]:
titanic_clear_columns['Sex'] = titanic_clear_columns['Sex'].replace(['malef'],'male')
titanic_clear_columns['Sex'] = titanic_clear_columns['Sex'].replace(['mal'],'male')
titanic_clear_columns['Sex'] = titanic_clear_columns['Sex'].replace(['fem'],'female')
titanic_clear_columns['Sex'] = titanic_clear_columns['Sex'].replace(['femmale'],'female')
unkalna_plec = titanic_clear_columns['Sex'].unique()
print(unkalna_plec)

['male' 'female']


Kolejnym etapem będzie uzupełnienie brakujących lub błędnych danych w kolumnie **Fare**, na sposób podobny jak w przypadku kolumny Age.

In [106]:
for i, row in titanic_clear_columns.iterrows():
    try:
        titanic_clear_columns.at[i, 'Fare'] = float(row['Fare'])       
    except ValueError:
        titanic_clear_columns.at[i, 'Fare'] = np.NaN

titanic_clear_columns[['Fare']] = titanic_clear_columns[['Fare']].astype(float).round(2)
titanic_clear_columns.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked     object
dtype: object

W powyższym wyniku można zauważyć, że kolumna **Fare** zmieniła typ danych na liczbowy.

Kolejnym etapem będzie sprawdzenie, czy występują zduplikowane rekordy.

In [109]:
titanic_clear_columns.duplicated(keep='first')

PassengerId
1       False
2       False
3       False
4       False
5       False
        ...  
888     False
889     False
890     False
891     False
1000    False
Length: 892, dtype: bool

Jak widać duplikaty występują (np. dla pasażera o id 11). Kolejnym krokiem będzie ich usunięcie.

In [110]:
titanic_clear_columns.drop_duplicates(keep = False, inplace = True)
titanic_clear_columns.duplicated(keep='first')

PassengerId
1       False
2       False
3       False
4       False
5       False
        ...  
888     False
889     False
890     False
891     False
1000    False
Length: 887, dtype: bool

Wykonanie powyższych kroków pozwoli na oczyszczenie i przygotowanie do analizy większości zbiorów danych.<br>
Przygotowane w ten sposób dane pozwolą na dalszą analizę, budowę modeli dla zbioru oraz wykorzystanie uczenia maszynowego do przewidywania danych.

Ostatnim etapem sprawozdania jest eksport wyczyszczonego zbioru do pliku **TitanicCleaned.tsv**.

In [111]:
titanic_clear_columns.to_csv('TitanicCleaned.tsv', sep = '\t')